<a href="https://colab.research.google.com/github/ewakobrzynska/MachineLearning/blob/main/075Ensemble_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method.

In [57]:
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()

data_set = iris.data[0:len(iris.target)-20,:]
labels = iris.target[0:len(iris.target)-20]
unique_labels = np.unique(iris.target)

test_data_set = iris.data[-20:,:]
test_labels = iris.target[-20:]

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [58]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [65]:
# Define the base classifiers
base_classifiers = [
    ('lr', LinearRegression()),
    ('knn', KNeighborsClassifier()),
    ('svm', SVC()),
    ('dt', DecisionTreeClassifier()),
    ('nb', GaussianNB()),
    ('qda', QuadraticDiscriminantAnalysis())
]

def build_classifiers():
    classifiers = {}
    for name, clf in base_classifiers:
        classifiers[name] = clf
    return classifiers

In [76]:
from sklearn.linear_model import LinearRegression, LogisticRegression

def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers.values():
        classifier.fit(data_set, labels)
    for classifier in classifiers.values():
        output.append(classifier.predict(data_set))
    output = np.array(output).T
    stacked_classifier = LogisticRegression()
    stacked_classifier.fit(output, labels)
    test_output = []
    for classifier in classifiers.values():
        test_output.append(classifier.predict(test_data_set))
    test_output = np.array(test_output).T
    predicted = stacked_classifier.predict(test_output)
    return predicted


In [77]:
classifiers = build_classifiers()
predicted = build_stacked_classifier(classifiers)
accuracy = accuracy_score(test_labels, predicted)
print(accuracy)

0.95


## Exercise 2:

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [14]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

#z notatnika '074Ensemble_Stacking.ipynb'
def calculate_accuracy_vector(predicted, labels):
    result = []
    for i in range(len(predicted)):
        if predicted[i] == labels[i]:
            result.append(1)
        else:
            result.append(0)
    return result

Fill the two functions below:

In [52]:
def set_new_weights(model):
    # fill the code here (two lines)
    I = np.array(calculate_accuracy_vector(model.predict(test_set), test_labels))
    weights = (1 + I) / (1 + I).sum()
    return weights

Train the classifier with the code below:

In [53]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.00129786 0.00129786 0.00129786 0.00129786 0.00129786 0.00129786
 0.00129786 0.00064893 0.00064893 0.00129786 0.00129786 0.00129786
 0.00129786 0.00064893 0.00129786 0.00129786 0.00129786 0.00064893
 0.00129786 0.00064893 0.00064893 0.00129786 0.00064893 0.00129786
 0.00129786 0.00129786 0.00064893 0.00064893 0.00129786 0.00064893
 0.00129786 0.00064893 0.00064893 0.00129786 0.00129786 0.00064893
 0.00129786 0.00129786 0.00129786 0.00129786 0.00129786 0.00129786
 0.00064893 0.00064893 0.00129786 0.00064893 0.00064893 0.00129786
 0.00064893 0.00129786 0.00064893 0.00064893 0.00129786 0.00129786
 0.00064893 0.00064893 0.00129786 0.00064893 0.00064893 0.00129786
 0.00129786 0.00064893 0.00064893 0.00129786 0.00064893 0.00064893
 0.00129786 0.00064893 0.00129786 0.00129786 0.00129786 0.00129786
 0.00064893 0.00129786 0.00129786 0.00064893 0.00129786 0.00129786
 0.00129786 0.00064893 0.00129786 0.00129786 0.00129786 0.00064893
 0.00129786 0.00064893 0.00129786 0.00129786 0.00129786 0.0006

Set the validation data set:

In [54]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [55]:
def get_prediction(x):
    output = np.array([classifier.predict(x) for classifier in classifiers])
    predicted = np.argmax(np.bincount(output.flatten()))
    return [predicted]

Test it:

In [56]:
prediction = get_prediction(validate_x)[0]
print(prediction)

1
